In [1]:
import sys
print(f"{sys.version}")

3.9.4 (default, Apr  9 2021, 11:43:21) [MSC v.1916 64 bit (AMD64)]


In [2]:
from copy import deepcopy
from enum import Enum
from typing import List, Tuple, NamedTuple, Callable, Optional

Grid = List[List[int]]
Options = List[List[List[int]]]

GRID_SIZE: int = 9
SECTOR_SIZE: int = 3
VALUE_RANGE: int = 9
EMPTY = 0

class GridLocation(NamedTuple):
    row: int
    column: int



In [3]:
# sample puzzles
puz1 = [[5,1,7,6,0,0,0,3,4],
         [2,8,9,0,0,4,0,0,0],
         [3,4,6,2,0,5,0,9,0],
         [6,0,2,0,0,0,0,1,0],
         [0,3,8,0,0,6,0,4,7],
         [0,0,0,0,0,0,0,0,0],
         [0,9,0,0,0,0,0,7,8],
         [7,0,3,4,0,0,5,6,0],
         [0,0,0,0,0,0,0,0,0]]

inp2  = [[5,1,7,6,0,0,0,3,4],
         [0,8,9,0,0,4,0,0,0],
         [3,0,6,2,0,5,0,9,0],
         [6,0,0,0,0,0,0,1,0],
         [0,3,0,0,0,6,0,4,7],
         [0,0,0,0,0,0,0,0,0],
         [0,9,0,0,0,0,0,7,8],
         [7,0,3,4,0,0,5,6,0],
         [0,0,0,0,0,0,0,0,0]]

inpd  = [[1,0,5,7,0,2,6,3,8],
         [2,0,0,0,0,6,0,0,5],
         [0,6,3,8,4,0,2,1,0],
         [0,5,9,2,0,1,3,8,0],
         [0,0,2,0,5,8,0,0,9],
         [7,1,0,0,3,0,5,0,2],
         [0,0,4,5,6,0,7,2,0],
         [5,0,0,0,0,4,0,6,3],
         [3,2,6,1,0,7,0,0,4]]

hard  = [[8,5,0,0,0,2,4,0,0],
         [7,2,0,0,0,0,0,0,9],
         [0,0,4,0,0,0,0,0,0],
         [0,0,0,1,0,7,0,0,2],
         [3,0,5,0,0,0,9,0,0],
         [0,4,0,0,0,0,0,0,0],
         [0,0,0,0,8,0,0,7,0],
         [0,1,7,0,0,0,0,0,0],
         [0,0,0,0,3,6,0,4,0]]

diff  = [[0,0,5,3,0,0,0,0,0],
         [8,0,0,0,0,0,0,2,0],
         [0,7,0,0,1,0,5,0,0],
         [4,0,0,0,0,5,3,0,0],
         [0,1,0,0,7,0,0,0,6],
         [0,0,3,2,0,0,0,8,0],
         [0,6,0,5,0,0,0,0,9],
         [0,0,4,0,0,0,0,3,0],
         [0,0,0,0,0,9,7,0,0]]


In [4]:
class Sudoku():
    def __init__(self, initial_values: Grid = None, rows: int = GRID_SIZE, columns: int = GRID_SIZE ) -> None:
        self._rows: int = rows
        self._columns: int = columns
        if initial_values == None:
            self.grid: Grid = [[EMPTY for c in range(columns)] for r in range(rows)]
        elif len(initial_values) != rows:
            print(f"initial_values: {len(initial_values)} rows, should be {GRID_SIZE}.")
        elif not (all([len(r)==GRID_SIZE for r in initial_values])):
            print(f"initial_values: not all rows contain {GRID_SIZE} columns.")
        else:
            self.grid: Grid = initial_values.copy()
        self._options: Options = self.get_options()


    def __str__(self) -> str:
        first_row = True
        output: str = "["
        for row in self.grid:
            first_col = True
            if first_row:
                output += "["
                first_row = False
            else:
                output += " ["
            for col in row:
                if first_col:
                    output += str(col)
                    first_col = False
                else:
                    output += ", " + str(col)
            output += "],\n"
        output += "]\n"
        return output


    def is_valid(self) -> bool:
        """
        checks if sudoku constraints are satisfied
        """
        for r in range(self._rows):
            for c in range(self._columns):
                if self.grid[r][c] != EMPTY:
                    # check row
                    for j in range(self._columns):
                        if (c != j) & (self.grid[r][c] == self.grid[r][j]):
                            return False
                    # check column
                    for i in range(self._rows):
                        if (r != i) & (self.grid[r][c] == self.grid[i][c]):
                            return False
                    # check sector
                    sec_r = r // SECTOR_SIZE
                    sec_c = c // SECTOR_SIZE
                    for i in range(SECTOR_SIZE):
                        for j in range(SECTOR_SIZE):
                            si = SECTOR_SIZE*sec_r + i
                            sj = SECTOR_SIZE*sec_c + j
                            if (r != si) & (c != sj) & (self.grid[r][c] == self.grid[si][sj]):
                                return False
        return True


    def _init_domain(self) -> Options:
        """
        initializes a list array of 1 through 9 domain options for each cell
        """
        domain = []
        for r in range(self._rows):
            row = []
            for c in range(self._columns):
                row.append(list(range(1,VALUE_RANGE+1)))
            domain.append(row)
        return domain


    def get_options(self) -> Options:
        """
        given a sudoku puzzle grid with values entered, an array of options for each cell is returned  
        """
        opts = self._init_domain()
        for r in range(self._rows):
            for c in range(self._columns):
                if self.grid[r][c] != EMPTY:
                    opts[r][c] = []
                else:
                    # check row
                    for j in range(self._columns):
                        if self.grid[r][j] in opts[r][c]:
                            opts[r][c].remove(self.grid[r][j])
                    # check column
                    for i in range(self._rows):
                        if self.grid[i][c] in opts[r][c]:
                            opts[r][c].remove(self.grid[i][c])
                    # check sector
                    sec_r = r // SECTOR_SIZE
                    sec_c = c // SECTOR_SIZE
                    for i in range(SECTOR_SIZE):
                        for j in range(SECTOR_SIZE):
                            si = SECTOR_SIZE*sec_r + i
                            sj = SECTOR_SIZE*sec_c + j
                            if self.grid[si][sj] in opts[r][c]:
                                opts[r][c].remove(self.grid[si][sj])
        return opts


    def set_cell(self, rcv: Optional[Tuple[int, int, int]] ):
        """
        set a sudoku puzzle grid cell (r, c) to value (v)
        """
        if rcv:
            r, c, v = rcv
            self.grid[r][c] = v
            self._options: Options = self.get_options()
        else:
            print(f"in set_cell() rcv is {rcv}.")
        return self

    
    def all_cells_filled(self):
        """
        check if all cells are filled
        Returns: True if no empty cells exist, False if empty cells remain
        """
        for r in range(self._rows):
            for c in range(self._columns):
                if self.grid[r][c] == EMPTY:
                    return False
        return True
    

    def print_num_options(self):
        """
        prints a grid conaining number of options for each cell
        """
        for r in range(self._rows):
            print([len(self._options[r][c]) for c in range(self._columns)])


    def num_options_all_zero(self) -> bool:
        """
        if any option counts greater than zero, returns False
        """
        for r in range(self._rows):
            for c in range(self._columns):
                if len(self._options[r][c]) > 0:
                    return False
        return True


    def get_first_option(self, num: int) -> Optional[Tuple[int, int, int]]:
        """
        find first cell with only num options
            return row , column and the single option
        otherwise return None
        """
        for r in range(self._rows):
            for c in range(self._columns):
                if len(self._options[r][c]) == num:
                    return (r, c, self._options[r][c][0])
        return None


    def fill_easy(self):
        """
        fill first cell with one option
        repeat until all cells with a single option are filled
        """
        d = True
        step_count = 0
        opt = self.get_first_option(1)
        while (opt is not None):
            self.set_cell(opt)
            step_count += 1
            #print(f"Step: {step_count}\n{self}")
            opt = self.get_first_option(1)
        return self


In [5]:
p1 = Sudoku(deepcopy(puz1))

In [6]:
p1

In [7]:
print(p1)

[[5, 1, 7, 6, 0, 0, 0, 3, 4],
 [2, 8, 9, 0, 0, 4, 0, 0, 0],
 [3, 4, 6, 2, 0, 5, 0, 9, 0],
 [6, 0, 2, 0, 0, 0, 0, 1, 0],
 [0, 3, 8, 0, 0, 6, 0, 4, 7],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 7, 8],
 [7, 0, 3, 4, 0, 0, 5, 6, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
]



In [8]:
p1.grid[1][2]

9

In [9]:
p1._rows

9

In [10]:
p1._columns

9

In [11]:
p1.grid

[[5, 1, 7, 6, 0, 0, 0, 3, 4],
 [2, 8, 9, 0, 0, 4, 0, 0, 0],
 [3, 4, 6, 2, 0, 5, 0, 9, 0],
 [6, 0, 2, 0, 0, 0, 0, 1, 0],
 [0, 3, 8, 0, 0, 6, 0, 4, 7],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 7, 8],
 [7, 0, 3, 4, 0, 0, 5, 6, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [12]:
p1._options

[[[], [], [], [], [8, 9], [8, 9], [2, 8], [], []],
 [[], [], [], [1, 3, 7], [1, 3, 7], [], [1, 6, 7], [5], [1, 5, 6]],
 [[], [], [], [], [1, 7, 8], [], [1, 7, 8], [], [1]],
 [[],
  [5, 7],
  [],
  [3, 5, 7, 8, 9],
  [3, 4, 5, 7, 8, 9],
  [3, 7, 8, 9],
  [3, 8, 9],
  [],
  [3, 5, 9]],
 [[1, 9], [], [], [1, 5, 9], [1, 2, 5, 9], [], [2, 9], [], []],
 [[1, 4, 9],
  [5, 7],
  [1, 4, 5],
  [1, 3, 5, 7, 8, 9],
  [1, 2, 3, 4, 5, 7, 8, 9],
  [1, 2, 3, 7, 8, 9],
  [2, 3, 6, 8, 9],
  [2, 5, 8],
  [2, 3, 5, 6, 9]],
 [[1, 4],
  [],
  [1, 4, 5],
  [1, 3, 5],
  [1, 2, 3, 5, 6],
  [1, 2, 3],
  [1, 2, 3, 4],
  [],
  []],
 [[], [2], [], [], [1, 2, 8, 9], [1, 2, 8, 9], [], [], [1, 2, 9]],
 [[1, 4, 8],
  [2, 5, 6],
  [1, 4, 5],
  [1, 3, 5, 7, 8, 9],
  [1, 2, 3, 5, 6, 7, 8, 9],
  [1, 2, 3, 7, 8, 9],
  [1, 2, 3, 4, 9],
  [2],
  [1, 2, 3, 9]]]

In [13]:
p1.is_valid()

True

In [14]:
p1.print_num_options()

[0, 0, 0, 0, 2, 2, 2, 0, 0]
[0, 0, 0, 3, 3, 0, 3, 1, 3]
[0, 0, 0, 0, 3, 0, 3, 0, 1]
[0, 2, 0, 5, 6, 4, 3, 0, 3]
[2, 0, 0, 3, 4, 0, 2, 0, 0]
[3, 2, 3, 6, 8, 6, 5, 3, 5]
[2, 0, 3, 3, 5, 3, 4, 0, 0]
[0, 1, 0, 0, 4, 4, 0, 0, 3]
[3, 3, 3, 6, 8, 6, 5, 1, 4]


In [15]:
p1.num_options_all_zero()

False

In [16]:
o1 = p1.get_first_option(1)
o1

(1, 7, 5)

In [17]:
print(o1[0], o1[1])
p1._options[o1[0]][o1[1]]

1 7


[5]

In [18]:
p1.set_cell(p1.get_first_option(1))

In [19]:
p1.grid

[[5, 1, 7, 6, 0, 0, 0, 3, 4],
 [2, 8, 9, 0, 0, 4, 0, 5, 0],
 [3, 4, 6, 2, 0, 5, 0, 9, 0],
 [6, 0, 2, 0, 0, 0, 0, 1, 0],
 [0, 3, 8, 0, 0, 6, 0, 4, 7],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 7, 8],
 [7, 0, 3, 4, 0, 0, 5, 6, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [20]:
p1._options

[[[], [], [], [], [8, 9], [8, 9], [2, 8], [], []],
 [[], [], [], [1, 3, 7], [1, 3, 7], [], [1, 6, 7], [], [1, 6]],
 [[], [], [], [], [1, 7, 8], [], [1, 7, 8], [], [1]],
 [[],
  [5, 7],
  [],
  [3, 5, 7, 8, 9],
  [3, 4, 5, 7, 8, 9],
  [3, 7, 8, 9],
  [3, 8, 9],
  [],
  [3, 5, 9]],
 [[1, 9], [], [], [1, 5, 9], [1, 2, 5, 9], [], [2, 9], [], []],
 [[1, 4, 9],
  [5, 7],
  [1, 4, 5],
  [1, 3, 5, 7, 8, 9],
  [1, 2, 3, 4, 5, 7, 8, 9],
  [1, 2, 3, 7, 8, 9],
  [2, 3, 6, 8, 9],
  [2, 8],
  [2, 3, 5, 6, 9]],
 [[1, 4],
  [],
  [1, 4, 5],
  [1, 3, 5],
  [1, 2, 3, 5, 6],
  [1, 2, 3],
  [1, 2, 3, 4],
  [],
  []],
 [[], [2], [], [], [1, 2, 8, 9], [1, 2, 8, 9], [], [], [1, 2, 9]],
 [[1, 4, 8],
  [2, 5, 6],
  [1, 4, 5],
  [1, 3, 5, 7, 8, 9],
  [1, 2, 3, 5, 6, 7, 8, 9],
  [1, 2, 3, 7, 8, 9],
  [1, 2, 3, 4, 9],
  [2],
  [1, 2, 3, 9]]]

In [21]:
p1.print_num_options()

[0, 0, 0, 0, 2, 2, 2, 0, 0]
[0, 0, 0, 3, 3, 0, 3, 0, 2]
[0, 0, 0, 0, 3, 0, 3, 0, 1]
[0, 2, 0, 5, 6, 4, 3, 0, 3]
[2, 0, 0, 3, 4, 0, 2, 0, 0]
[3, 2, 3, 6, 8, 6, 5, 2, 5]
[2, 0, 3, 3, 5, 3, 4, 0, 0]
[0, 1, 0, 0, 4, 4, 0, 0, 3]
[3, 3, 3, 6, 8, 6, 5, 1, 4]


In [22]:
p1.fill_easy()

In [23]:
p1.num_options_all_zero()

True

In [24]:
p1.is_valid()

True

In [37]:
def sudoku_solver(sudoku: Sudoku) -> Optional[Sudoku]:
    sud1 = deepcopy(sudoku)
    sud1._options = sud1.get_options()
    sud1 = sud1.fill_easy()
    sud1._options = sud1.get_options()
    print("input:\n", sud1)
    
    if sud1.is_valid() & sud1.all_cells_filled():
        return sud1

    for n in range(2, VALUE_RANGE):
        opt = sud1.get_first_option(n)
        if opt is None:
            continue
        opt_list: List = sud1._options[opt[0]][opt[1]]
        print(sud1._options)
        for v in opt_list:
            print("try", (opt[0],opt[1],v))
            sud1 = sud1.set_cell((opt[0],opt[1],v)).fill_easy()
            if (sud1 is not None):
                if sud1.is_valid():
                    print("call solver")
                    solution = sudoku_solver(sud1)
                    print("soln", solution)
                    if (solution is not None):
                        if solution.is_valid() & solution.all_cells_filled():
                            return solution
    return None


In [38]:
print(puz1)

[[5, 1, 7, 6, 0, 0, 0, 3, 4], [2, 8, 9, 0, 0, 4, 0, 0, 0], [3, 4, 6, 2, 0, 5, 0, 9, 0], [6, 0, 2, 0, 0, 0, 0, 1, 0], [0, 3, 8, 0, 0, 6, 0, 4, 7], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 9, 0, 0, 0, 0, 0, 7, 8], [7, 0, 3, 4, 0, 0, 5, 6, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [39]:
p1 = Sudoku(puz1)
print(p1)
soln1 = sudoku_solver(p1)
print(soln1)
print("is_valid:", soln1.is_valid())
print("all_cells_filled", soln1.all_cells_filled())

[[5, 1, 7, 6, 0, 0, 0, 3, 4],
 [2, 8, 9, 0, 0, 4, 0, 0, 0],
 [3, 4, 6, 2, 0, 5, 0, 9, 0],
 [6, 0, 2, 0, 0, 0, 0, 1, 0],
 [0, 3, 8, 0, 0, 6, 0, 4, 7],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 7, 8],
 [7, 0, 3, 4, 0, 0, 5, 6, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
]

input:
 [[5, 1, 7, 6, 9, 8, 2, 3, 4],
 [2, 8, 9, 1, 3, 4, 7, 5, 6],
 [3, 4, 6, 2, 7, 5, 8, 9, 1],
 [6, 7, 2, 8, 4, 9, 3, 1, 5],
 [1, 3, 8, 5, 2, 6, 9, 4, 7],
 [9, 5, 4, 7, 1, 3, 6, 8, 2],
 [4, 9, 5, 3, 6, 2, 1, 7, 8],
 [7, 2, 3, 4, 8, 1, 5, 6, 9],
 [8, 6, 1, 9, 5, 7, 4, 2, 3],
]

[[5, 1, 7, 6, 9, 8, 2, 3, 4],
 [2, 8, 9, 1, 3, 4, 7, 5, 6],
 [3, 4, 6, 2, 7, 5, 8, 9, 1],
 [6, 7, 2, 8, 4, 9, 3, 1, 5],
 [1, 3, 8, 5, 2, 6, 9, 4, 7],
 [9, 5, 4, 7, 1, 3, 6, 8, 2],
 [4, 9, 5, 3, 6, 2, 1, 7, 8],
 [7, 2, 3, 4, 8, 1, 5, 6, 9],
 [8, 6, 1, 9, 5, 7, 4, 2, 3],
]

is_valid: True
all_cells_filled True


In [40]:
p2 = Sudoku(inp2)
print(p2)
soln2 = sudoku_solver(p2)
print(soln2)
print("is_valid:", soln2.is_valid())
print("all_cells_filled:", soln2.all_cells_filled())

[[5, 1, 7, 6, 0, 0, 0, 3, 4],
 [0, 8, 9, 0, 0, 4, 0, 0, 0],
 [3, 0, 6, 2, 0, 5, 0, 9, 0],
 [6, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 3, 0, 0, 0, 6, 0, 4, 7],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 7, 8],
 [7, 0, 3, 4, 0, 0, 5, 6, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
]

input:
 [[5, 1, 7, 6, 0, 0, 2, 3, 4],
 [2, 8, 9, 0, 0, 4, 7, 5, 6],
 [3, 4, 6, 2, 7, 5, 8, 9, 1],
 [6, 0, 0, 0, 0, 0, 3, 1, 0],
 [0, 3, 0, 0, 0, 6, 9, 4, 7],
 [0, 0, 0, 0, 0, 0, 6, 8, 0],
 [0, 9, 0, 0, 0, 0, 0, 7, 8],
 [7, 2, 3, 4, 0, 0, 5, 6, 9],
 [0, 0, 0, 0, 0, 0, 0, 2, 3],
]

[[[], [], [], [], [8, 9], [8, 9], [], [], []], [[], [], [], [1, 3], [1, 3], [], [], [], []], [[], [], [], [], [], [], [], [], []], [[], [5, 7], [2, 4, 5, 8], [5, 7, 8, 9], [2, 4, 5, 8, 9], [2, 7, 8, 9], [], [], [2, 5]], [[1, 8], [], [1, 2, 5, 8], [1, 5, 8], [1, 2, 5, 8], [], [], [], []], [[1, 4, 9], [5, 7], [1, 2, 4, 5], [1, 3, 5, 7, 9], [1, 2, 3, 4, 5, 9], [1, 2, 3, 7, 9], [], [], [2, 5]], [[1, 4], [], [1, 4, 5], [1, 3, 5], [1, 2, 3, 5, 6], 

AttributeError: 'NoneType' object has no attribute 'is_valid'

In [29]:
p3 = Sudoku(inpd)
print(p3)
soln3 = sudoku_solver(p3)
print(soln3)
print("is_valid:", soln3.is_valid())
print("all_cells_filled:", soln3.all_cells_filled())

[[1, 0, 5, 7, 0, 2, 6, 3, 8],
 [2, 0, 0, 0, 0, 6, 0, 0, 5],
 [0, 6, 3, 8, 4, 0, 2, 1, 0],
 [0, 5, 9, 2, 0, 1, 3, 8, 0],
 [0, 0, 2, 0, 5, 8, 0, 0, 9],
 [7, 1, 0, 0, 3, 0, 5, 0, 2],
 [0, 0, 4, 5, 6, 0, 7, 2, 0],
 [5, 0, 0, 0, 0, 4, 0, 6, 3],
 [3, 2, 6, 1, 0, 7, 0, 0, 4],
]

[[1, 4, 5, 7, 9, 2, 6, 3, 8],
 [2, 8, 7, 3, 1, 6, 4, 9, 5],
 [9, 6, 3, 8, 4, 5, 2, 1, 7],
 [4, 5, 9, 2, 7, 1, 3, 8, 6],
 [6, 3, 2, 4, 5, 8, 1, 7, 9],
 [7, 1, 8, 6, 3, 9, 5, 4, 2],
 [8, 9, 4, 5, 6, 3, 7, 2, 1],
 [5, 7, 1, 9, 2, 4, 8, 6, 3],
 [3, 2, 6, 1, 8, 7, 9, 5, 4],
]

soln3 True


In [36]:
p4 = Sudoku(hard)
print(p4)
soln4 = sudoku_solver(p4)
print(soln4)
print("is_valid:", soln4.is_valid())
print("all_cells_filled:", soln4.all_cells_filled())

[[8, 5, 0, 0, 0, 2, 4, 0, 0],
 [7, 2, 0, 0, 0, 0, 0, 0, 9],
 [0, 0, 4, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 7, 0, 0, 2],
 [3, 0, 5, 0, 0, 0, 9, 0, 0],
 [0, 4, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 8, 0, 0, 7, 0],
 [0, 1, 7, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 3, 6, 0, 4, 0],
]

call solver
call solver
call solver
call solver
call solver
call solver
call solver
call solver
call solver
call solver
call solver
call solver
call solver
call solver
call solver
call solver
call solver
[[8, 5, 6, 9, 7, 2, 4, 1, 3],
 [7, 2, 3, 6, 1, 4, 5, 8, 9],
 [1, 9, 4, 8, 5, 3, 6, 2, 7],
 [6, 8, 9, 1, 4, 7, 3, 5, 2],
 [3, 7, 5, 2, 0, 8, 9, 6, 1],
 [2, 4, 1, 3, 6, 5, 7, 0, 8],
 [4, 3, 2, 5, 8, 9, 1, 7, 6],
 [5, 1, 7, 4, 2, 0, 8, 3, 0],
 [9, 0, 8, 7, 3, 6, 2, 4, 5],
]

soln4 True


In [37]:
p5 = Sudoku(diff)
print(p5)
soln5 = sudoku_solver(p5)
print(soln5)
print("is_valid:", soln5.is_valid())
print("all_cells_filled:", soln5.all_cells_filled())

[[0, 0, 5, 3, 0, 0, 0, 0, 0],
 [8, 0, 0, 0, 0, 0, 0, 2, 0],
 [0, 7, 0, 0, 1, 0, 5, 0, 0],
 [4, 0, 0, 0, 0, 5, 3, 0, 0],
 [0, 1, 0, 0, 7, 0, 0, 0, 6],
 [0, 0, 3, 2, 0, 0, 0, 8, 0],
 [0, 6, 0, 5, 0, 0, 0, 0, 9],
 [0, 0, 4, 0, 0, 0, 0, 3, 0],
 [0, 0, 0, 0, 0, 9, 7, 0, 0],
]

call solver
call solver
call solver
call solver
call solver
call solver
call solver
call solver
call solver
[[6, 4, 5, 3, 2, 7, 1, 9, 8],
 [8, 3, 1, 9, 5, 4, 6, 2, 7],
 [9, 7, 2, 6, 1, 8, 5, 4, 3],
 [4, 8, 6, 1, 9, 5, 3, 7, 2],
 [2, 1, 9, 8, 7, 3, 4, 5, 6],
 [7, 5, 3, 2, 4, 6, 9, 8, 1],
 [3, 6, 7, 5, 8, 2, 0, 1, 9],
 [5, 9, 4, 7, 6, 1, 2, 3, 0],
 [1, 2, 8, 4, 3, 9, 7, 6, 5],
]

soln5 True


In [96]:
def is_valid(self) -> bool:
    """
    checks if sudoku constraints are satisfied
    """
    for r in range(self._rows):
        for c in range(self._columns):
            if self.grid[r][c] != EMPTY:
                # check row
                for j in range(self._columns):
                    if (c != j) & (self.grid[r][c] == self.grid[r][j]):
                        return False
                # check column
                for i in range(self._rows):
                    if (r != i) & (self.grid[r][c] == self.grid[i][c]):
                        return False
                # check sector
                sec_r = r // SECTOR_SIZE
                sec_c = c // SECTOR_SIZE
                for i in range(SECTOR_SIZE):
                    for j in range(SECTOR_SIZE):
                        si = SECTOR_SIZE*sec_r + i
                        sj = SECTOR_SIZE*sec_c + j
                        if (r != si) & (c != sj) & (self.grid[r][c] == self.grid[si][sj]):
                            return False
    return True


In [98]:
print("soln1", is_valid(soln1))
print("soln2", is_valid(soln2))


soln1 True
soln2 False
